In [1]:
%load_ext autoreload
%autoreload 2
# %env CUDA_VISIBLE_DEVICES=

In [2]:
import sys; sys.path.append("../")

In [3]:
from fof.encdec import EncoderDecoderModel
from fof.dataloader import ScicapDataModule
from pathlib import Path
import wandb

In [4]:
run = wandb.init(project="figuring-out-figures")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: figuring-out-figures (use `wandb login --relogin` to force relogin)


In [14]:
import pytorch_lightning as pl

datamodule = ScicapDataModule(
    "First-Sentence",
    batch_size=16,
    tokenizer=None,
    num_workers=32,
    root=Path("../scicap_data"),
    caption_type="normalized")
trainer = pl.Trainer(gpus=1)

/home/tetris/.virtualenvs/figuring-out-figures-Gm0t6Smt/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Initializing SCICAP training dataset
Initializing SCICAP testing dataset
Initializing SCICAP validation dataset
Time taken: 0.04568004608154297


In [8]:
def test_model(artifact_str: str = None, ckpt_path: str = None, use_test = False, **kwargs):
    if artifact_str is not None:
        artifact = run.use_artifact(artifact_str, type="model")
        artifact_dir = artifact.download()
        ckpt_path = Path(artifact_dir) / "model.ckpt"
    model = EncoderDecoderModel.load_from_checkpoint(ckpt_path, **kwargs)
    if use_test:
        trainer.test(model, datamodule=datamodule)
    else:
        trainer.validate(model, datamodule=datamodule)

In [6]:
# Text features only, best model
# test_model(ckpt_path="/data/kevin/arxiv/evaluation_checkpoints/text-features-only.ckpt")

In [13]:
# Image features (CLIP) only, original caption, best model
test_model(artifact_str="figuring-out-figures/figuring-out-figures/model-yw5qm3wp:v16", use_test=True)


wandb: Downloading large artifact model-yw5qm3wp:v16, 2112.90MB. 1 files... Done. 0:0:0
Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.4.self_attn.q_proj.bias', 'text_model.encoder.layers.5.mlp.fc2.weight', 'text_model.encoder.layers.6.self_attn.v_proj.bias', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.10.layer_norm2.weight', 'text_model.encoder.layers.5.mlp.fc1.bias', 'text_model.encoder.layers.5.self_attn.out_proj.bias', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.7.layer_norm1.weight', 'text_model.final_layer_norm.weight', 'text_model.encoder.layers.1.mlp.fc1.bias', 'text_model.encoder.layers.4.self_attn.k_proj.bias', 'text_model.encoder.layers.0.self_attn.v_proj.weight', 'text_model.encoder.layers.1.self_attn.k_proj.weight', 'text_model.encoder.layers.4.self_attn.out_proj.bias', 'text_model.encoder.layers.1.m

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test/bleu_score        1.0177823305130005
        test/loss           1.2709388732910156
     test/perplexity        3.7739639282226562
   test/rouge_L_score       0.13340324459859038
    test/rouge_score        0.14704097289598733
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [15]:
# Image features (CLIP), DistilGPT, normalized captions, SCIBERT encoder
# for references, title, and abstract, best model
test_model(
    artifact_str='figuring-out-figures/figuring-out-figures/model-1d7mntmw:v2', use_test=True, use_top_p_sampling=True)


wandb: Downloading large artifact model-1d7mntmw:v2, 2532.30MB. 1 files... Done. 0:0:0
Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.4.self_attn.q_proj.bias', 'text_model.encoder.layers.5.mlp.fc2.weight', 'text_model.encoder.layers.6.self_attn.v_proj.bias', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.10.layer_norm2.weight', 'text_model.encoder.layers.5.mlp.fc1.bias', 'text_model.encoder.layers.5.self_attn.out_proj.bias', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.7.layer_norm1.weight', 'text_model.final_layer_norm.weight', 'text_model.encoder.layers.1.mlp.fc1.bias', 'text_model.encoder.layers.4.self_attn.k_proj.bias', 'text_model.encoder.layers.0.self_attn.v_proj.weight', 'text_model.encoder.layers.1.self_attn.k_proj.weight', 'text_model.encoder.layers.4.self_attn.out_proj.bias', 'text_model.encoder.layers.1.ml

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test/bleu_score        2.2109172344207764
        test/loss           0.9752202033996582
     test/perplexity        2.7556872367858887
   test/rouge_L_score       0.17755812889340514
    test/rouge_score        0.20243289611149098
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
# Image features (CLIP), DistilGPT, original captions, SCIBERT encoder
# for references, title, and abstract, best model
test_model(
    artifact_str='figuring-out-figures/figuring-out-figures/model-27py12gz:v2', use_test=True, use_top_p_sampling=False)


wandb: Downloading large artifact model-27py12gz:v2, 2532.30MB. 1 files... Done. 0:0:0
Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.4.self_attn.q_proj.bias', 'text_model.encoder.layers.5.mlp.fc2.weight', 'text_model.encoder.layers.6.self_attn.v_proj.bias', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.10.layer_norm2.weight', 'text_model.encoder.layers.5.mlp.fc1.bias', 'text_model.encoder.layers.5.self_attn.out_proj.bias', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.7.layer_norm1.weight', 'text_model.final_layer_norm.weight', 'text_model.encoder.layers.1.mlp.fc1.bias', 'text_model.encoder.layers.4.self_attn.k_proj.bias', 'text_model.encoder.layers.0.self_attn.v_proj.weight', 'text_model.encoder.layers.1.self_attn.k_proj.weight', 'text_model.encoder.layers.4.self_attn.out_proj.bias', 'text_model.encoder.layers.1.ml

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test/bleu_score         4.915057182312012
        test/loss           1.0891406536102295
     test/perplexity        3.1011884212493896
   test/rouge_L_score       0.2566786774732374
    test/rouge_score        0.2758900403293233
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
# DistilGPT top p caption
test_model(artifact_str="figuring-out-figures/figuring-out-figures/model-27py12gz:v1", use_test=True)